In [1]:
import time
import json
import datetime
import globus_sdk

from globus_sdk import TimerJob
from globus_compute_sdk import Executor
from globus_sdk.experimental.globus_app import UserApp

from globus_sdk.utils import slash_join

In [2]:
CLIENT_ID = "1b3a354b-30b2-415d-a137-751c23cf9a1f"
my_app = UserApp("crocus-user-app", client_id=CLIENT_ID)

my_app.add_scope_requirements({'flows': ["https://auth.globus.org/scopes/eec9b274-0c81-4334-bdc2-54e90e689b9a/all"]})

my_app.run_login_flow()

flows_client = globus_sdk.FlowsClient(app=my_app)


Please authenticate with Globus here:
-------------------------------------
https://auth.globus.org/v2/oauth2/authorize?client_id=1b3a354b-30b2-415d-a137-751c23cf9a1f&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+https%3A%2F%2Fauth.globus.org%2Fscopes%2Feec9b274-0c81-4334-bdc2-54e90e689b9a%2Fall&state=_default&response_type=code&code_challenge=pz0rPwyl9oG3b0x4UGQPziBNdjK_jn8tcaazwjn37JM&code_challenge_method=S256&access_type=online&prefill_named_grant=crocus-user-app
-------------------------------------



Enter the resulting Authorization Code here:
 vjLdg4MKv2vK7RqziuxNdZR58DALLM


In [3]:
compute_endpoint = "a93bab84-bc75-43a0-8ab9-ba7a41a1a2d4"

In [4]:
wxt_function = "d78ce48d-614a-4c77-add7-1c0e25c3472d"

In [5]:
gce = Executor(endpoint_id=compute_endpoint)

In [8]:
# Prepare payload for ESGF ingest-wxt
wxt_data = {
    "ndays": 1,
    "y": 2024,
    "m": 8,
    "d": 2,
    "site": 'NU',
    "hours": 1,
    "odir": "/home/rchard/src/CROCUS/output/"
}

# Start the task
future = gce.submit_to_registered_function(wxt_function, kwargs=wxt_data)

In [9]:
# Wait and print the result
result = future.result()
print(result)

['/home/rchard/src/CROCUS/output//crocus-NU-wxt-a1_20240802_000000.nc']


In [34]:
flow_definition = {
    "Comment": "A CROCUS WXT flow",
    "StartAt": "TransferInput",
    "States": {
        "TransferInput": {
            "Comment": "Transfer input data",
            "Type": "Action",
            "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
            "Parameters": {
                "source_endpoint_id.$": "$.input.source.id",
                "destination_endpoint_id.$": "$.input.destination.id",
                "transfer_items": [
                    {
                        "source_path.$": "$.input.source.path",
                        "destination_path.$": "$.input.destination.path",
                        "recursive": True,
                    }
                ]
            },
            "ResultPath": "$.TransferFiles",
            "WaitTime": 300,
            "Next": "ProcessWXT"
        },
        "ProcessWXT": {
            "Comment": "Collect WXT data from Sage",
            "Type": "Action",
            "ActionUrl": "https://compute.actions.globus.org/",
            "Parameters": {
                "endpoint.$": "$.input.compute_endpoint",
                "function.$": "$.input.wxt_function",
                "kwargs.$": "$.input.wxt_kwargs"
            },
            "ResultPath": "$.CROCUS_output",
            "WaitTime": 600,
            "End": True
        },
    }
}

In [12]:
flow_id = "b944990b-1e87-4401-98fd-fd9caab491fc"

In [46]:
# flow = flows_client.create_flow(title="CROCUS Flow", definition=flow_definition, input_schema={})
flow = flows_client.update_flow(flow_id=flow_id, title="CROCUS Flow", definition=flow_definition, input_schema={})

In [47]:
flow_input = {
    "input": {
        "source": {
            "id": "6c54cade-bde5-45c1-bdea-f4bd71dba2cc",
            "path": "/home/share/godata/"
        },
        "destination": {
            "id": "31ce9ba0-176d-45a5-add3-f37d233ba47d",
            "path": "/~/test/"
        },
        "compute_endpoint": compute_endpoint,
        "wxt_kwargs": wxt_data,
        "wxt_function": wxt_function,
    }
}

In [48]:
flow_id = flow['id']

In [49]:
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    app=my_app,
)

In [50]:
run = specific_flow_client.run_flow(
  body=flow_input,
  label="CROCUS Example",
  tags=['CROCUS', 'example']
)

In [51]:
# Get run details
# run = flows_client.get_run(run_id)

run_id = run['run_id']
run_status = run['status']
print("This flow can be monitored in the Web App:")
print(f"https://app.globus.org/runs/{run_id}")
print(f"Flow run started with ID: {run_id} - Status: {run_status}")

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')
    
# Run completed
print(json.dumps(run.data, indent=2))

This flow can be monitored in the Web App:
https://app.globus.org/runs/e0ef8255-ecab-4f87-a30d-569084d39e56
Flow run started with ID: e0ef8255-ecab-4f87-a30d-569084d39e56 - Status: ACTIVE
Run status: INACTIVE
{
  "run_id": "e0ef8255-ecab-4f87-a30d-569084d39e56",
  "flow_id": "b944990b-1e87-4401-98fd-fd9caab491fc",
  "flow_title": "CROCUS Flow",
  "flow_last_updated": "2024-10-02T19:21:52.699432+00:00",
  "start_time": "2024-10-02T19:22:49.894445+00:00",
  "status": "INACTIVE",
  "display_status": "INACTIVE",
  "details": {
    "code": "ConsentRequired",
    "state_name": "TransferInput",
    "description": "Flow run requires user intervention to proceed. For state 'TransferInput': Missing required data_access consent",
    "required_scope": "https://auth.globus.org/scopes/b944990b-1e87-4401-98fd-fd9caab491fc/flow_b944990b_1e87_4401_98fd_fd9caab491fc_user[*https://auth.globus.org/scopes/actions.globus.org/transfer/transfer[*urn:globus:auth:scope:transfer.api.globus.org:all[*https://auth

# Add an input schema

This gives us the "Start" button on the flows GUI.

In [45]:
input_schema = {
    "required": [
        "input"
    ],
    "properties": {
        "input": {
            "type": "object",
            "required": [
                "source",
                "destination",
                "compute_endpoint",
                "wxt_function",
                "wxt_kwargs"
            ],
            "properties": {
                "source": {
                    "type": "object",
                    "title": "Select source collection and path",
                    "description": "The source collection and path (path MUST end with a slash)",
                    "format": "globus-collection",
                    "required": [
                        "id",
                        "path"
                    ],
                    "properties": {
                        "id": {
                            "type": "string",
                            "format": "uuid"
                        },
                        "path": {
                            "type": "string"
                        }
                    },
                    "additionalProperties": False
                },
                "destination": {
                    "type": "object",
                    "title": "Select destination collection and path",
                    "description": "The destination collection and path (path MUST end with a slash); default collection is 'Globus Tutorials on ALCF Eagle'",
                    "format": "globus-collection",
                    "required": [
                        "id",
                        "path"
                    ],
                    "properties": {
                        "id": {
                            "type": "string",
                            "format": "uuid"
                        },
                        "path": {
                            "type": "string"
                        }
                    },
                    "additionalProperties": False
                },
                "compute_endpoint": {
                    "type": "string",
                    "format": "uuid",
                    "default": compute_endpoint,
                    "title": "Globus Compute Endpoint ID",
                    "description": "The UUID of the Globus Compute endpoint where the function will run"
                },
                "wxt_function": {
                    "type": "string",
                    "format": "uuid",
                    "default": wxt_function,
                    "title": "Globus Compute Function ID",
                    "description": "The UUID of the function to invoke; must be registered with the Globus Compute service"
                },
                "wxt_kwargs": {
                    "type": "object",
                    "title": "Function Inputs",
                    "description": "Inputs to pass to the function",
                    "properties":  {
                        "ndays": {
                            "type": "integer",
                            "default": 1
                        },
                        "y": {
                            "type": "integer",
                            "default": 2024
                        },
                        "m": {
                            "type": "integer",
                            "default": 8
                        },
                        "d": {
                            "type": "integer",
                            "default": 1
                        },
                        "site": {
                            "type": "string",
                            "default": "NU"
                        },
                        "hours": {
                            "type": "integer",
                            "default": 1
                        },
                        "odir": {
                            "type": "string",
                            "default": "/home/rchard/src/CROCUS/output/"
                        },
                    },
                    "additionalProperties": False
                }
            },
            "additionalProperties": False
        }
    },
    "additionalProperties": False
}

In [ ]:
flow = flows_client.update_flow(flow_id=flow_id, title="CROCUS Flow", definition=flow_definition, input_schema=input_schema)

# Configure a Timer

This will automate the invocation of the flow each day.

In [ ]:
from globus_sdk.scopes import TimerScopes, FlowsScopes

In [ ]:
timer_client = globus_sdk.TimerClient(app=my_app, app_scopes=FlowsScopes.all)

In [ ]:
callback_url = slash_join(specific_flow_client.base_url, f"/flows/{flow_id}/run")

In [ ]:
callback_url

In [ ]:
specific_flow_client.scopes.user

In [ ]:
specific_flow_client.scopes.url_scope_string('user')

In [ ]:
specific_flow_client.scopes.user

In [ ]:
timer = TimerJob(
    callback_url=callback_url,
    callback_body={"body": flow_input, "label": "CROCUS Timer Flow"},
    start=datetime.datetime.now(),
    interval=datetime.timedelta(seconds=600),
    name="CROCUS Flow Timer",
    scope=specific_flow_client.scopes.user,
    # scope=globus_sdk.SpecificFlowClient(flow_id).scopes.url_scope_string('user'),
)

In [ ]:
response = timer_client.create_job(timer)

In [ ]:
from globus_sdk import RefreshTokenAuthorizer
from globus_sdk import AccessTokenAuthorizer
from globus_sdk import NativeAppAuthClient
from globus_sdk import SpecificFlowClient
from globus_sdk import TimerClient
from globus_sdk.scopes.data import TimerScopes
_TIMER_CLIENT_UUID: str = "524230d7-ea86-4a52-8312-86065a9e0417"

In [ ]:
_REDIRECT_URI = "https://auth.globus.org/v2/web/auth-code"

def authenticate(client: NativeAppAuthClient, scope: str):
    """Perform Globus Authentication."""

    # assert False, scope
    client.oauth2_start_flow(
        redirect_uri=_REDIRECT_URI, refresh_tokens=True, requested_scopes=scope
    )

    url = client.oauth2_get_authorize_url()
    print("Please visit the following url to authenticate:")
    print(url)

    auth_code = input("Enter the auth code:")
    auth_code = auth_code.strip()
    return client.oauth2_exchange_code_for_tokens(auth_code)

def create_authorizer(flow_id, auth_type="timer"):
    client, specific_flow_scope = create_context(flow_id)

    if auth_type == "timer":
        resource_server = _TIMER_CLIENT_UUID
    else:
        resource_server = flow_id

    timer_scope = TimerScopes.make_mutable("timer")
    scopes = [timer_scope]
    # assert False, flows_scope

    sfc = SpecificFlowClient(flow_id=flow_id)
    specific_flow_scope_name = f"flow_{flow_id.replace('-', '_')}_user"
    specific_flow_scope = sfc.scopes.url_scope_string(specific_flow_scope_name)
    scopes.append(sfc.scopes.user)
    timer_scope.add_dependency(specific_flow_scope)

    print(timer_scope)
    
    tokens = authenticate(client, scopes)
    
    if auth_type == "flows":
        assert fid is not None
        flows_access_token = tokens.by_resource_server[fid]["access_token"]
        authorizer = AccessTokenAuthorizer(access_token=flows_access_token)

    elif auth_type == "timer":
        timer_access_token = tokens.by_resource_server[_TIMER_CLIENT_UUID][
            "access_token"
        ]
        authorizer = AccessTokenAuthorizer(access_token=timer_access_token)

    return authorizer, specific_flow_scope

def create_context(flow_id):
    sfc = SpecificFlowClient(flow_id=flow_id)
    specific_flow_scope_name = f"flow_{flow_id.replace('-', '_')}_user"
    specific_flow_scope = sfc.scopes.url_scope_string(specific_flow_scope_name)
    client = NativeAppAuthClient(client_id=CLIENT_ID)
    return client, specific_flow_scope


In [ ]:
authorizer, specific_flow_scope = create_authorizer(flow_id)

In [ ]:
specific_flow_scope

In [ ]:
timer = TimerJob(
    callback_url=callback_url,
    callback_body={"body": flow_input, "label": "CROCUS Timer Flow"},
    start=datetime.datetime.utcnow(),
    interval=datetime.timedelta(seconds=600),
    name="CROCUS Flow Timer",
    scope=specific_flow_scope,
    # scope=globus_sdk.SpecificFlowClient(flow_id).scopes.url_scope_string('user'),
)

In [ ]:
timer_client = TimerClient(authorizer=authorizer, app_name="CROCUS Timer")

In [ ]:
response = timer_client.create_job(timer)

In [ ]:
response

In [ ]:
timer_client.list_jobs()

In [ ]:
response.get('job_id')

In [ ]:
timer_client.get_job(response.get('job_id')).data

In [ ]:
timer_client.delete_job(response.get('job_id'))

In [ ]:
my_app = UserApp("crocus-user-app", client_id=CLIENT_ID)

In [ ]:
from globus_sdk.scopes.data import TimerScopes

# flow_scope = SpecificFlowClient(flow_id).scopes.user
# dep_scope = f"{TimerScopes.timer}[{flow_scope}]"


timer_scope = TimerScopes.make_mutable("timer")
timer_scope.add_dependency(flow_scope)


timer_client = globus_sdk.TimerClient(app=my_app, app_scopes=dep_scope)

In [ ]:
timer_client = globus_sdk.TimerClient(app=my_app, app_scopes=dep_scope)

In [ ]:
timer = TimerJob(
    callback_url=callback_url,
    callback_body={"body": flow_input, "label": "CROCUS Timer Flow"},
    start=datetime.datetime.utcnow(),
    interval=datetime.timedelta(seconds=600),
    name="CROCUS Flow Timer",
    scope=SpecificFlowClient(flow_id).scopes.user,
)

In [ ]:
SpecificFlowClient(flow_id).scopes.user

In [ ]:
response = timer_client.create_job(timer)

In [ ]:
response.get('job_id')

In [ ]:
timer_client.get_job(response.get('job_id')).data

In [ ]:
timer_client.delete_job(response.get('job_id'))